# Skipping the Passport Queue

--- Day 4: Passport Processing ---
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:
```
byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
```
Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:
```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

## Second part

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:
```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```
Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:
```
byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
```

Here are some invalid passports:
```
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
```
Here are some valid passports:
```
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```
Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [1]:
import os
import time
import math
import numpy as np
import re

In [2]:
RUN_WITH = 'input' ## admitted values: sample input sample2
SAMPLE_DATA = '''1 ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm
 
  

2 iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929


3 hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

4 hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in'''
FIELDS = '''byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)'''


SAMPLE_DATA_2='''
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007

pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719

'''
OPTIONALS = ['cid']
START = (1,1)

In [3]:
start = time.time()

In [4]:
day = 'Day_04'
file_end = '_01.txt'
input_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "Inputs")
input_path_day = os.path.join(input_path, day)
file_path = os.path.join(input_path_day, day+ file_end)

### Read input

In [5]:
if RUN_WITH == 'input':
    with open(file_path,'r') as f:
        values = f.read().split('\n\n')
elif RUN_WITH == 'sample':
    values =  SAMPLE_DATA.split('\n\n')
elif RUN_WITH == 'sample2':
    values =  SAMPLE_DATA_2.split('\n\n')   
else:
    print("No valid input selected")
values = [v.strip().replace('\n',' ') for v in values]

In [6]:
fields_needed = [f[0:3] for f in FIELDS.split('\n')]

fields_needed = list(filter(lambda x: x not in OPTIONALS, fields_needed))

In [7]:
values[0:5]

['eyr:2021 hgt:168cm hcl:#fffffd pid:180778832 byr:1923 ecl:amb iyr:2019 cid:241',
 'hcl:#341e13 ecl:lzr eyr:2024 iyr:2014 pid:161cm byr:1991 cid:59 hgt:150cm',
 'iyr:2018 eyr:2027 hgt:153cm pid:642977294 ecl:gry hcl:#c0946f byr:1999',
 'pid:#534f2e eyr:2022 ecl:amb cid:268 iyr:2028 hcl:2b079f byr:2008 hgt:185cm',
 'byr:2000 hgt:161cm ecl:blu eyr:2030 pid:647793597 hcl:#a97842 iyr:2011']

In [8]:
def has_all_fields(entry, fields_needed):
    for field in fields_needed:
        if not field +':' in entry:
            return False
    return True

In [9]:
hold_values = np.ones(len(values))

In [10]:
outcome = list(map(has_all_fields, values, [fields_needed] * len(values)))

### Output Part I

In [11]:
np.sum(outcome * hold_values)

260.0

part 2:

## validation rules:
```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```

In [12]:
# Validation rules
def byr(value: str) -> bool:
    return (len(value) == 4) and value.isdigit() and (1920 <= int(value) <= 2002)
def iyr(value: str) -> bool:
    return (len(value) == 4) and value.isdigit() and (2010 <= int(value) <= 2020)
def eyr(value: str) -> bool:
    return (len(value) == 4) and value.isdigit() and (2020 <= int(value) <= 2030)
def hgt(value: str) -> bool:
    hgt_type = None
    if value.endswith('cm'):
        hgt_type = 'cm'
    elif value.endswith('in'):
        hgt_type = 'in'
    if not hgt_type:
        return False
    if hgt_type:
        hgt = value.replace(hgt_type, '')
        if not hgt.isdigit():
            return False
        if hgt_type == 'cm':
            return 150 <= int(hgt) <= 193
        else:
            return 59 <= int(hgt) <= 76
    return False
def hcl(value: str) -> bool:
    if re.match('#[0-9a-f]{6}', value):
        return True
    return False
def ecl(value: str) -> bool:
    return value in 'amb blu brn gry grn hzl oth'.split()
def pid(value: str) -> bool:
    return value.isdigit() and len(value)==9
def cid(value: str) -> bool:
    return False

In [13]:
# Dispatch table:
functions = {
    'byr': byr,
    'iyr': iyr,
    'eyr': eyr,
    'hgt': hgt,
    'hcl': hcl,
    'ecl': ecl,
    'pid': pid,
    'cid': cid
}

In [14]:
#Sample record
values[0]

'eyr:2021 hgt:168cm hcl:#fffffd pid:180778832 byr:1923 ecl:amb iyr:2019 cid:241'

In [15]:
#Remove empty records if still present
values = list(filter(lambda x: x != '',values))

In [16]:
#Prepare each record for code verification
vals = list(map(str.split, values))
vals[0]

['eyr:2021',
 'hgt:168cm',
 'hcl:#fffffd',
 'pid:180778832',
 'byr:1923',
 'ecl:amb',
 'iyr:2019',
 'cid:241']

In [17]:
tests_passed = []
for passport in vals:
    #Pass the first part of the code (e.g. 'hgt') to the dictionary, pass the second part (e.g. 168cm) as the argument
    #Sum all trues for each record
    tests_passed.append(sum([functions[code.split(':')[0]](code.split(':')[1]) for code in passport]))

### Output Part II

In [18]:
len(list(filter(lambda x: x == 7, tests_passed)))

153

In [19]:
end = time.time()
end - start

0.12213301658630371